In [1]:
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
import shutil

from scipy.stats import gamma
from math import ceil
from numpy.fft import fft, ifft, ifftshift

# first use qa to get sub namelist for filter

In [2]:
os.getcwd()

'C:\\Users\\73707\\Desktop\\fmri\\ode'

In [8]:
#first extract name list pass qa
qa = pd.read_csv("abcd_sst_func_qa_pass.txt", sep = ",")
sub = qa.iloc[:, 1:].reset_index(drop = True)
run1 = sub[sub['run'] == 1]
name_ls_run1 = run1['sub'].tolist()
name_ls_run1

['NDARINV699886DJ',
 'NDARINVVC2R39EB',
 'NDARINVKNHZYKRL',
 'NDARINV6ARV4JZP',
 'NDARINV7UTXNP58',
 'NDARINV6VUN0VRU',
 'NDARINVALLKP7M5',
 'NDARINV18KWP8N1',
 'NDARINVTV25YAN3',
 'NDARINVLJ3FUY7X',
 'NDARINV5WTABLWY',
 'NDARINVCKTT8HAT',
 'NDARINVEU6NMEWP',
 'NDARINVMKFGRBV4',
 'NDARINV5NHWRD1C',
 'NDARINVEBMR55XD',
 'NDARINV1PHTC329',
 'NDARINVV3Z0YVUT',
 'NDARINV0RHDH1EL',
 'NDARINV4G3XF9GX',
 'NDARINVV8URWFJ6',
 'NDARINVEVN2CFJL',
 'NDARINVBZ9H2FLC',
 'NDARINVWNKZ5Z4L',
 'NDARINVWNUXY3KF',
 'NDARINV3MLPX0CG',
 'NDARINVFB2YC0F5',
 'NDARINVU8N8L34K',
 'NDARINVKHXDKCB7',
 'NDARINVPF3UJTKW',
 'NDARINV3E2U4BWJ',
 'NDARINVLW8Z7G7E',
 'NDARINV9PJ7VRDA',
 'NDARINVHXB99NBE',
 'NDARINV73ZVLFTC',
 'NDARINVJ51UCKWX',
 'NDARINVY98NL5JN',
 'NDARINVDPYR4KM2',
 'NDARINVZMVK0FMA',
 'NDARINV7ZPBL89M',
 'NDARINVEXAKNZ18',
 'NDARINVF65R9VTJ',
 'NDARINVHTH8AYJK',
 'NDARINV52AU0VED',
 'NDARINVB9WLDALN',
 'NDARINV725NP8B2',
 'NDARINV3ADKZJ2Z',
 'NDARINVU9DAJLMN',
 'NDARINV5N1N1DEL',
 'NDARINVWP2U8M5Y',


In [9]:
len(name_ls_run1)

6462

In [10]:
run2 = sub[sub['run'] == 2]
name_ls_run2 = run2['sub'].to_list()
name_ls_run2

1        NDARINVVC2R39EB
3        NDARINVKNHZYKRL
6        NDARINV6ARV4JZP
8        NDARINV7UTXNP58
9        NDARINV6VUN0VRU
              ...       
12740    NDARINVX7RUVFDV
12742    NDARINV28VRHGXD
12745    NDARINVNE8U2H3X
12746    NDARINV64HHWR5T
12749    NDARINVF3Z2K76G
Name: sub, Length: 6288, dtype: object

In [12]:
name_ls = list(set(name_ls_run1) & set(name_ls_run2))

In [11]:
len(name_ls_run2)

6288

In [13]:
len(name_ls)

5936

In [16]:
np.save("namelist.npy",name_ls)

In [77]:
#then extract abcd subjects based on name_ls
os.chdir('abcd')
for fname in os.listdir():
    pat_name = fname[4:19]
    if pat_name in name_ls:
        shutil.copy(fname,"../abcd_filter/")

In [13]:
os.chdir("..")

# Then calculate singular matrix to fliter one more step

In [78]:
os.chdir("../abcd_filter")

In [79]:
def get_data_dataframe(filename):
    df = pd.read_csv(filename, sep='\t', header=None)
    df = df.iloc[:,1:].loc[:, 1::2].to_numpy().astype(np.float32) ### for odds 
    #df = df.iloc[:,1:].loc[:, 2::2].to_numpy().astype(np.float32) ### for even 
    return df

In [80]:
for fname in os.listdir():
    df = get_data_dataframe(fname)
    #calculate singular matrix
    try:
        np.linalg.cholesky([np.cov(preprocessing.StandardScaler().fit(df).transform(df).T)])
        shutil.copy(fname,"../abcd_nosig/")
    except: np.linalg.LinAlgError: ("matrix is singular" )    

In [102]:
nosig_ls = []
for fname in os.listdir('abcd_deconv'):
    nosig_ls.append(fname[0:15])

In [70]:
name_ls = []
for fname in os.listdir("events2"):
    name_ls.append(fname[:-12] + "1" + fname[-11:])    

In [73]:
for fname in os.listdir("events"):
    if fname in name_ls:
        shutil.copy("events/"+fname, "event_small/")

In [76]:
dummy

,correct_Go,correct_Stop,incorrect_Stop
0,0,0,0
1,1,0,0
2,0,1,0
3,1,0,0
4,1,0,0
...,...,...,...
176,1,0,0
177,1,0,0
178,1,0,0
179,0,1,0


In [77]:
#for events, short folder names, do one-hot encoding
for fname in os.listdir("event_small"):
    
    df = pd.read_csv("event_small/"+fname, sep = '\t', header = 0)
    row_idx = np.floor(df.onset/0.8).astype('int')
    reshape_DF(df)
    dummy = pd.get_dummies(df['trial_type'])
    dummy.drop(['else'], axis = 1, inplace=True)
    dim1 = row_idx.values[-1] + 1
    dim2 = dummy.shape[1]
    U = pd.DataFrame(np.zeros((dim1, dim2)))
    for i in range(len(row_idx)):
        row_num = row_idx[i]
        U.iloc[row_num, :] = dummy.iloc[i, :].tolist()

    U.to_csv(f"train_event/{fname[4:19]}.csv")

In [33]:
os.chdir("..")

In [2]:
def gammaHRF(TR, paras=None, len_of_seconds=32, onset_seconds=0):
    
    if paras is None:
        paras = [6,16,1,1,6]

    dt = TR/16
    u = np.array(range(0, ceil(len_of_seconds/dt))) - ceil(onset_seconds/dt)
    
    tmp1 = gamma.pdf(u, a = paras[0]/paras[2], scale = 1/(dt/paras[2])) 
    tmp2 = gamma.pdf(u, a = paras[1]/paras[3], scale = 1/(dt/paras[3]))/paras[4]       
    hrf = tmp1 - tmp2

    hrf = hrf[np.array(range(0, ceil(len_of_seconds/TR)))*16 + 1]

    hrf = hrf/sum(hrf)

    return hrf

def wiener_deconvolution(signal, kernel, lambd = 1e-3):
    
    kernel = np.hstack((kernel, np.zeros(len(signal) - len(kernel)))) # zero pad the kernel to same length
    H = fft(kernel)
    deconvolved = np.real(ifft(fft(signal)*np.conj(H)/(H*np.conj(H) + lambd**2)))
    return deconvolved

In [3]:
hrf = gammaHRF(0.8)
hrf

array([ 2.37770533e-09,  1.51693529e-03,  1.87869621e-02,  6.09300625e-02,
        1.12455733e-01,  1.51843887e-01,  1.68017823e-01,  1.61912696e-01,
        1.40861833e-01,  1.13084890e-01,  8.47761883e-02,  5.94567129e-02,
        3.85051384e-02,  2.20017616e-02,  9.43707481e-03,  1.52142450e-04,
       -6.45746830e-03, -1.08884673e-02, -1.35468651e-02, -1.47817288e-02,
       -1.49097159e-02, -1.42239355e-02, -1.29908693e-02, -1.14421537e-02,
       -9.76702463e-03, -8.10870144e-03, -6.56555260e-03, -5.19623985e-03,
       -4.02729227e-03, -3.06150608e-03, -2.28590503e-03, -1.67847472e-03,
       -1.21333240e-03, -8.64330329e-04, -6.07295824e-04, -4.21203239e-04,
       -2.88584821e-04, -1.95451099e-04, -1.30934520e-04, -8.68098314e-05])

In [82]:
os.chdir('../abcd_nosig')

In [85]:
fname = os.listdir()[0]
ls = get_data_dataframe(fname)
df = pd.DataFrame(ls)
df

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,131.791458,221.895935,75.203072,125.823944,161.678238,232.707886,161.873184,264.367279,235.946686,363.513397,...,335.259003,334.564209,261.734375,336.619568,329.705383,262.986115,241.724579,276.643036,299.971069,352.225037
1,116.219528,195.118607,67.192947,109.084869,148.390808,214.120712,150.926376,244.729279,212.378708,331.569580,...,281.661530,282.312866,224.418259,288.834686,283.019897,224.494278,195.793808,231.508621,246.496628,291.923462
2,110.448067,185.872513,63.920254,102.634995,143.551544,205.523163,146.058487,235.606873,202.700195,316.592651,...,264.861176,262.740448,213.946579,272.427917,269.346863,214.088013,180.432999,216.629684,228.451141,271.687592
3,108.241493,182.563171,62.928268,100.427467,141.327133,201.246445,143.915955,231.047485,199.275314,309.949005,...,258.781250,252.169540,210.362137,267.021423,264.057617,210.668869,173.661621,211.798569,222.577759,268.044128
4,106.973579,180.307068,62.356945,99.636978,140.222809,199.641418,142.553589,228.302460,196.957748,306.759216,...,256.713715,247.791580,209.520096,264.195526,262.669037,210.486084,171.973633,210.237915,220.340759,263.977448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,106.265999,177.797440,61.626549,98.471764,137.367950,196.513336,140.765640,222.644455,194.542877,301.902008,...,253.405853,239.764175,207.663559,263.742981,261.709229,209.252563,168.661896,208.260178,218.150177,260.467010
441,106.367317,177.339737,61.939186,98.179764,136.984421,195.873550,139.789658,221.541870,194.434692,301.437378,...,252.386307,240.133835,207.822372,262.847076,261.599335,209.221771,169.170898,207.968170,218.061478,260.426331
442,106.129898,177.547760,61.563148,97.893181,137.191345,194.672958,139.882278,220.824646,193.822617,300.310364,...,251.729050,241.029419,207.969391,262.812653,261.837433,207.997147,168.511719,207.498672,217.510818,262.527130
443,106.147751,177.719116,61.594093,97.422356,137.301941,194.421387,140.119781,219.692307,194.107681,300.550720,...,251.163498,240.516953,208.148453,262.548004,261.307037,209.454285,167.848755,207.086014,217.516846,260.684509


#  deconv and normalization

In [103]:
for fname in os.listdir():
    file = get_data_dataframe(fname)
    df = pd.DataFrame(file)
    
    ##apply deconvolution
    nrows, ncols = df.shape
    df_decov = pd.DataFrame(np.zeros((nrows, ncols)))
    for i in range(ncols):
        df_decov.iloc[:, i] = wiener_deconvolution(df.iloc[:, i], hrf)
    #normalization
    decov_norm = preprocessing.StandardScaler().fit(df_decov).transform(df_decov)
    decov_norm_df = pd.DataFrame(decov_norm)
    name = fname[4:19]
    decov_norm_df.to_csv(f"../abcd_deconv/{name}.csv", index = False)

#  adjust event for homogeneous case

In [5]:
os.chdir("../event_small")

In [38]:
for fname in os.listdir():
    event = pd.read_csv(fname)
    event = event.iloc[:, 1:]
    col_sum = event.sum(axis=0)
    event_bin = (col_sum > 0).astype(np.int_)
    event_bin2 = pd.DataFrame((event_bin)).T
    event_bin2.to_csv("../event_small_homogenous/"+fname)

# deconv and normalization on testset

In [2]:
def get_data_dataframe(filename):
    df = pd.read_csv(filename, sep='\t', header=None)
    df = df.iloc[:,1:].loc[:, 1::2].to_numpy().astype(np.float32) ### for odds 
    #df = df.iloc[:,1:].loc[:, 2::2].to_numpy().astype(np.float32) ### for even 
    return df

In [24]:
def gammaHRF(TR, paras=None, len_of_seconds=32, onset_seconds=0):
    
    if paras is None:
        paras = [6,16,1,1,6]

    dt = TR/16
    u = np.array(range(0, ceil(len_of_seconds/dt))) - ceil(onset_seconds/dt)
    
    tmp1 = gamma.pdf(u, a = paras[0]/paras[2], scale = 1/(dt/paras[2])) 
    tmp2 = gamma.pdf(u, a = paras[1]/paras[3], scale = 1/(dt/paras[3]))/paras[4]       
    hrf = tmp1 - tmp2

    hrf = hrf[np.array(range(0, ceil(len_of_seconds/TR)))*16 + 1]

    hrf = hrf/sum(hrf)

    return hrf

def wiener_deconvolution(signal, kernel, lambd = 1e-3):
    
    kernel = np.hstack((kernel, np.zeros(len(signal) - len(kernel)))) # zero pad the kernel to same length
    H = fft(kernel)
    deconvolved = np.real(ifft(fft(signal)*np.conj(H)/(H*np.conj(H) + lambd**2)))
    return deconvolved

hrf = gammaHRF(0.8)

In [36]:
for fname in os.listdir("abcd_run2_small"):
    file = get_data_dataframe("abcd_run2_small/" + fname)
    df = pd.DataFrame(file)
    #deconv
    nrows, ncols = df.shape
    deconv_df = pd.DataFrame(np.zeros((nrows, ncols)))
    for i in range(ncols):
        deconv_df.iloc[:,i] = wiener_deconvolution(df.iloc[:,i], hrf)
    #normalization
    deconv_norm = preprocessing.StandardScaler().fit(deconv_df).transform(deconv_df)
    decov_norm_df = pd.DataFrame(deconv_norm)
    name = fname[4:19]
    decov_norm_df.to_csv(f"abcd_run2_deconv/{name}.csv", index = False)

In [43]:
filename = os.listdir("events2")[2]
df = pd.read_csv("events2/"+filename, sep='\t', header=0)
df

,onset,duration,trial_type
0,0.00,6.40,dummy
1,8.52,0.47,correct_go
2,10.51,0.42,correct_go
3,12.29,0.45,correct_go
4,14.07,0.44,correct_go
...,...,...,...
176,343.08,0.80,noresp_go
177,344.81,0.35,correct_go
178,346.51,0.02,correct_stop
179,348.48,0.32,incorrect_go


In [49]:
def reshape_DF(df):    
    for i in range(df.shape[0]):
        if df.iloc[i, 2] == "correct_go":
            df.iloc[i, 2] = "correct_Go"
        elif df.iloc[i, 2] == "correctlate_go":
            df.iloc[i, 2] = "correct_Go"
        elif df.iloc[i, 2] == "incorrect_go":
            df.iloc[i, 2] = "incorrect_Go"
        elif df.iloc[i, 2] == "incorrectlate_go":
            df.iloc[i, 2] = "incorrect_Go"
        elif df.iloc[i, 2] == "correct_stop":
            df.iloc[i, 2] = "correct_Stop"
        elif df.iloc[i, 2] == "incorrect_stop":
            df.iloc[i, 2] = "incorrect_Stop"
        else:
            df.iloc[i, 2] = "else"

In [50]:
reshape_DF(df)
df.iloc[:, 2].value_counts()

correct_Go        104
incorrect_Go       39
incorrect_Stop     25
else                9
correct_Stop        4
Name: trial_type, dtype: int64

In [40]:
np.floor(df.onset/0.8).astype('int')

0        0
1       10
2       13
3       15
4       17
      ... 
176    428
177    431
178    433
179    435
180    437
Name: onset, Length: 181, dtype: int32

In [52]:
row_idx = np.floor(df['onset']/0.8).astype("int")
row_idx.values[-1] + 1

438

In [55]:
dim1 = row_idx.values[-1] + 1
dim2 = 4
U = pd.DataFrame(np.zeros((dim1, dim2)))
dummy = pd.get_dummies(df['trial_type'])
dummy.drop(['else'], axis = 1, inplace=True)
dummy

,correct_Go,correct_Stop,incorrect_Go,incorrect_Stop
0,0,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
176,0,0,0,0
177,1,0,0,0
178,0,1,0,0
179,0,0,1,0


In [59]:
df

,onset,duration,trial_type
0,0.00,6.40,else
1,8.51,0.80,else
2,10.38,0.02,correct_Stop
3,12.56,0.50,correct_Go
4,14.34,0.54,correct_Go
...,...,...,...
176,339.16,0.80,correct_Go
177,341.28,0.80,correct_Go
178,342.83,0.80,else
179,344.55,0.02,incorrect_Stop


In [58]:
df.shape

(181, 3)

In [ ]:
#extract events
for fname in os.listdir("events2"):
    df = pd.read_csv("events2/"+fname, sep = '\t')
    row_idx = np.floor(df.onset/0.8).astype('int')
    dim1 = row_idx.values[-1] + 1
    dim2 = 4
    U = pd.DataFrame(np.zeros((dim1, dim2)))
    dummy = pd.get_dummies(df['trial_type'])
    for i in range(len(row_idx)):
        row_num = row_idx[i]
        U.iloc[row_num, :] = dummy.iloc[i, :].tolist()

    U.to_csv(f"event2_short/{fname[4:19]}.csv")

In [18]:
df.iloc[:, 2].value_counts()

correct_Go        104
incorrect_Go       39
incorrect_Stop     25
else                9
correct_Stop        4
Name: trial_type, dtype: int64

In [57]:
#extract events
for fname in os.listdir("events2"):
    df = pd.read_csv("events2/"+fname, sep = '\t', header = 0)
    row_idx = np.floor(df.onset/0.8).astype('int')
    reshape_DF(df)
    dim1 = row_idx.values[-1] + 1
    dim2 = 4
    U = pd.DataFrame(np.zeros((dim1, dim2)))
    dummy = pd.get_dummies(df['trial_type'])
    dummy.drop(['else'], axis = 1, inplace=True)
    for i in range(len(row_idx)):
        row_num = row_idx[i]
        U.iloc[row_num, :] = dummy.iloc[i, :].tolist()

    U.to_csv(f"event2_small/{fname[4:19]}.csv")